# NUSANTAMART
 *(Nusantara market) Website UMKM dengan Sistem Rekomendasi*

## Data Understanding

In [3]:
# Import library
import pandas as pd
import numpy as np 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from pathlib import Path
import matplotlib.pyplot as plt

Data Loading

In [4]:
food = pd.read_csv('/indonesian_food.csv')
food

,foodId,Nama,Tipe
0,1,Sosis Bakar,ayam-daging
1,2,Ngohiong Ayam Udang,ayam-daging
2,3,Rawon Ayam,ayam-daging
3,4,Usus Goreng Crispy,ayam-daging
4,5,Ceker Rica Rica,ayam-daging
...,...,...,...
1268,1269,Es Cincau,buah-minuman
1269,1270,Asinan Rambutan,buah-minuman
1270,1271,Asinan Buah,buah-minuman
1271,1272,Sop Buah,buah-minuman


##**Data Preprocessing**

melihat informasi pada data

In [9]:
food.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1273 entries, 0 to 1272
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   foodId  1273 non-null   int64 
 1   Nama    1273 non-null   object
 2   Tipe    1273 non-null   object
dtypes: int64(1), object(2)
memory usage: 30.0+ KB


In [10]:
food.head()

,foodId,Nama,Tipe
0,1,Sosis Bakar,ayam-daging
1,2,Ngohiong Ayam Udang,ayam-daging
2,3,Rawon Ayam,ayam-daging
3,4,Usus Goreng Crispy,ayam-daging
4,5,Ceker Rica Rica,ayam-daging


In [11]:
food.describe()

,foodId
count,1273.000000
mean,637.000000
std,367.627756
min,1.000000
25%,319.000000
50%,637.000000
75%,955.000000
max,1273.000000


##**Data Preparation**

Memeriksa Missing Value Pada dataset

In [12]:
food.isnull().sum()

foodId    0
Nama      0
Tipe      0
dtype: int64

In [ ]:
len(food)

1273

In [13]:
# Mengurutkan makanan berdasarkan id kemudian memasukkannya ke dalam variabel fix_food
fix_food = food.sort_values('foodId', ascending=True)
fix_food

,foodId,Nama,Tipe
0,1,Sosis Bakar,ayam-daging
1,2,Ngohiong Ayam Udang,ayam-daging
2,3,Rawon Ayam,ayam-daging
3,4,Usus Goreng Crispy,ayam-daging
4,5,Ceker Rica Rica,ayam-daging
...,...,...,...
1268,1269,Es Cincau,buah-minuman
1269,1270,Asinan Rambutan,buah-minuman
1270,1271,Asinan Buah,buah-minuman
1271,1272,Sop Buah,buah-minuman


In [14]:
# Mengecek berapa jumlah fix_food
len(fix_food.foodId.unique())

1273

In [15]:
# Mengonversi data series ‘foodId’ menjadi dalam bentuk list
food_id = fix_food['foodId'].tolist()
 
# Mengonversi data series ‘Nama’ menjadi dalam bentuk list
food_nama = fix_food['Nama'].tolist()
 
# Mengonversi data series ‘Tipe’ menjadi dalam bentuk list
food_tipe = fix_food['Tipe'].tolist()

 
print(len(food_id))
print(len(food_nama))
print(len(food_tipe))

1273
1273
1273


In [16]:
# Membuat dictionary untuk data ‘foodId’, ‘Nama’, ‘tipe’.
food_new = pd.DataFrame({
    'foodId': food_id,
    'name': food_nama,
    'tipe': food_tipe,
})
food_new

,foodId,name,tipe
0,1,Sosis Bakar,ayam-daging
1,2,Ngohiong Ayam Udang,ayam-daging
2,3,Rawon Ayam,ayam-daging
3,4,Usus Goreng Crispy,ayam-daging
4,5,Ceker Rica Rica,ayam-daging
...,...,...,...
1268,1269,Es Cincau,buah-minuman
1269,1270,Asinan Rambutan,buah-minuman
1270,1271,Asinan Buah,buah-minuman
1271,1272,Sop Buah,buah-minuman


In [17]:
data = food_new
data.sample(5)

,foodId,name,tipe
202,203,Koloke Ayam,ayam-daging
344,345,Gulai Ikan Tenggiri,ikan-seafood
22,23,Dumpling Ayam Kukus,ayam-daging
402,403,Telur Geprek,tahu-tempe-telur
1043,1044,Roti Boy O (Mocha Bun),kue-roti


TF-IDF Vectorizer

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
 
# Inisialisasi TfidfVectorizer
tf = TfidfVectorizer()
 
# Melakukan perhitungan idf pada data tipe
tf.fit(data['tipe']) 
 
# Mapping array dari fitur index integer ke fitur nama
tf.get_feature_names()

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['ayam',
 'bakso',
 'buah',
 'daging',
 'ikan',
 'jajanan',
 'jeli',
 'keripik',
 'kerupuk',
 'kue',
 'mie',
 'minuman',
 'nasi',
 'pasar',
 'pasta',
 'puding',
 'roti',
 'sambal',
 'sayur',
 'seafood',
 'sop',
 'soto',
 'tahu',
 'telur',
 'tempe']

In [19]:
# Melakukan fit lalu ditransformasikan ke bentuk matrix
tfidf_matrix = tf.fit_transform(data['tipe']) 
 
# Melihat ukuran matrix tfidf
tfidf_matrix.shape 

(1273, 25)

In [20]:
# Mengubah vektor tf-idf dalam bentuk matriks dengan fungsi todense()
tfidf_matrix.todense()

matrix([[0.70710678, 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.70710678, 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.70710678, 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        ...,
        [0.        , 0.        , 0.70710678, ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.70710678, ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.70710678, ..., 0.        , 0.        ,
         0.        ]])

In [21]:
# Membuat dataframe untuk melihat tf-idf matrix
# Kolom diisi dengan genre buku
# Baris diisi dengan judul buku
 
pd.DataFrame(
    tfidf_matrix.todense(), 
    columns=tf.get_feature_names(),
    index=data.name
).sample(22, axis=1).sample(10, axis=0)

,telur,pasar,daging,bakso,sayur,kerupuk,soto,tahu,mie,jeli,...,jajanan,keripik,buah,sop,ikan,minuman,seafood,nasi,puding,sambal
name,,,,,,,,,,,,,,,,,,,,,
Es Boba Brown Sugar,0.0,0.0,0.000000,0.00000,0.0,0.0,0.00000,0.0,0.0,0.0,...,0.0,0.0,0.707107,0.00000,0.000000,0.707107,0.000000,0.0,0.0,0.0
Patin Bakar,0.0,0.0,0.000000,0.00000,0.0,0.0,0.00000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.00000,0.707107,0.000000,0.707107,0.0,0.0,0.0
Cheese Cake Kukus (Tanpa Oven),0.0,0.0,0.000000,0.00000,0.0,0.0,0.00000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.0,0.0,0.0
Tom Yam Ayam,0.0,0.0,0.000000,0.57735,0.0,0.0,0.57735,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.57735,0.000000,0.000000,0.000000,0.0,0.0,0.0
Asinan Buah,0.0,0.0,0.000000,0.00000,0.0,0.0,0.00000,0.0,0.0,0.0,...,0.0,0.0,0.707107,0.00000,0.000000,0.707107,0.000000,0.0,0.0,0.0
Daging Goreng,0.0,0.0,0.707107,0.00000,0.0,0.0,0.00000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.0,0.0,0.0
Sandwich Buah,0.0,0.0,0.000000,0.00000,0.0,0.0,0.00000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.0,0.0,0.0
Otak Otak Bandeng,0.0,0.0,0.000000,0.00000,0.0,0.0,0.00000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.00000,0.707107,0.000000,0.707107,0.0,0.0,0.0
Ayam Goreng Cabe,0.0,0.0,0.707107,0.00000,0.0,0.0,0.00000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.0,0.0,0.0


In [22]:
from sklearn.metrics.pairwise import cosine_similarity
 
# Menghitung cosine similarity pada matrix tf-idf
cosine_sim = cosine_similarity(tfidf_matrix) 
cosine_sim

array([[1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 1., 1.],
       [0., 0., 0., ..., 1., 1., 1.],
       [0., 0., 0., ..., 1., 1., 1.]])

In [23]:
# Membuat dataframe dari variabel cosine_sim dengan baris dan kolom berupa tilte
cosine_sim_df = pd.DataFrame(cosine_sim, index=data['name'], columns=data['name'])
print('Shape:', cosine_sim_df.shape)
 
# Melihat similarity matrix pada setiap judul buku
cosine_sim_df.sample(5, axis=1).sample(10, axis=0)

Shape: (1273, 1273)


name,Kue Mangkok,Tahu Gejrot,Nasi Goreng Magelangan,Risoles Mayo,Rainbow Cake Kukus
name,,,,,
Sambal Kecap,0.0,0.0,0.0,0.0,0.0
Tom Yam Seafood Udang Ikan,0.0,0.0,0.0,0.0,0.0
Tumis Telur Tauge,0.0,0.0,0.0,0.0,0.0
Ikan Kerapu Asam Manis,0.0,0.0,0.0,0.0,0.0
Ayam Bakar Padang,0.0,0.0,0.0,0.0,0.0
Donat Crispy,0.0,0.0,0.0,0.0,1.0
Sup Udang,0.0,0.0,0.0,0.0,0.0
Rempeyek Kacang Tanah,1.0,0.0,0.0,1.0,0.0
Tumis Kacang Panjang,0.0,0.0,0.0,0.0,0.0


In [24]:
from os import name
def food_recommendations(name, similarity_data=cosine_sim_df, items=data[['name', 'tipe']], k=20):
    """
    Rekomendasi makanan berdasarkan kemiripan dataframe
 
    Parameter:
    ---
    name : tipe data string (str)
                nama makanan (index kemiripan dataframe)
    similarity_data : tipe data pd.DataFrame (object)
                      Kesamaan dataframe, simetrik, dengan nama sebagai 
                      indeks dan kolom
    items : tipe data pd.DataFrame (object)
            Mengandung kedua nama dan fitur lainnya yang digunakan untuk mendefinisikan kemiripan
    k : tipe data integer (int)
        Banyaknya jumlah rekomendasi yang diberikan
    ---
 
 
    Pada index ini, kita mengambil k dengan nilai similarity terbesar 
    pada index matrix yang diberikan (i).
    """
 
 
    # Mengambil data dengan menggunakan argpartition untuk melakukan partisi secara tidak langsung sepanjang sumbu yang diberikan    
    # Dataframe diubah menjadi numpy
    # Range(start, stop, step)
    index = similarity_data.loc[:,name].to_numpy().argpartition(
        range(-1, -k, -1))
    
    # Mengambil data dengan similarity terbesar dari index yang ada
    closest = similarity_data.columns[index[-1:-(k+2):-1]]
    
    # Drop name agar name yang dicari tidak muncul dalam daftar rekomendasi
    closest = closest.drop(name, errors='ignore')
 
    return pd.DataFrame(closest).merge(items).head(k)

In [25]:
data[data.name.eq('Kue Ketan Hitam Kukus')]

,foodId,name,tipe
1018,1019,Kue Ketan Hitam Kukus,kue-roti


In [26]:
food_recommendations('Kue Ketan Hitam Kukus')

,name,tipe
0,Rainbow Cake Kukus,kue-roti
1,Panada,kue-roti
2,Kue Telur Gabus,kue-roti
3,Donat Jadul,kue-roti
4,Hotang (Hotdog Kentang) / Sostang (Sosis Kentang),kue-roti
5,Ongol Ongol,kue-roti
6,Lemper Ayam Bakar,kue-roti
7,Onde-Onde Isi Kacang Hijau,kue-roti
8,Misro,kue-roti
9,Serabi Bandung Oncom,kue-roti
